<a href="https://colab.research.google.com/github/BBB-WU/NLP/blob/main/Bert_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import**

In [ ]:
pip install transformers

In [ ]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

In [ ]:
import pandas as pd
import numpy as np
import re

# **Data Preprocessing**

In [ ]:
df = pd.read_csv('/content/new_train(bare-miss).csv',encoding= 'unicode_escape')

In [ ]:
label_dict = {'barely-true': 'fake',
 'false': 'fake',
 'half-true': 'true',
 'mostly-true': 'true',
 'pants-fire': 'fake',
 'true': 'true',
 'no-flip':'true',
 'full-flop':'fake'}

df['label'] = df.label.replace(label_dict)

In [ ]:
pm = df.apply(sorted,key=pd.isnull)
 
df= pm[~pd.isnull(pm).all(1)].fillna('')

In [ ]:
df.value=list(np.int_(df.value))

In [ ]:
df.head()

,news,label,value
0,the cleveland police has issued a stand down o...,fake,0
1,president obama funneled $716 billion out of m...,fake,0
2,mr hurt would reopen that darn medicare doughn...,true,1
3,for 22 percent of nh meals on wheels recipient...,true,1
4,says his comment about extremists being a jv t...,fake,0


In [ ]:
df['label'].value_counts()

fake    11405
true    11405
Name: label, dtype: int64

In [ ]:
df['value'].value_counts()

1    11405
0    11405
Name: value, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.value.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.value.values)

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].news.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=64, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].news.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=64, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].value.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].value.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

# **Build the Model**

In [ ]:
bert = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=2,
                                                      output_attentions=False,
                                                      output_hidden_states=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class ConvBert(nn.Module):
    def __init__(self, 
                 bert,
                 num_labels,
                 n_filters,
                 filter_sizes,
                 dropout):
        super().__init__()
        self.num_labels = num_labels

        self.bert = bert

        embedding_dim = bert.config.to_dict()['hidden_size']

        self.convs = nn.ModuleList([
                        nn.Conv2d(in_channels = 1, 
                             out_channels = n_filters, 
                             kernel_size = (fs, embedding_dim)) 
                        for fs in filter_sizes
                                    ])

        self.dropout = nn.Dropout(dropout)

        self.classifier = nn.Linear(len(filter_sizes) * n_filters, num_labels)
        
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):

        
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        

        hidden_layers = outputs[2]
        #all hidden_states 

        embedded = hidden_layers[-1]
        #last hidden_state #embedded = [batch size, sent len, emb dim]

        embedded = embedded.unsqueeze(1)
        #embedded = [batch size, 1, sent len, emb dim]

        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
                
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))
        #cat = [batch size, n_filters * len(filter_sizes)]

        logits = self.classifier(cat)
        #conv_output = [batch size, num_labels]

        loss = None
        
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        

        return  loss,logits

In [ ]:

model = ConvBert(bert,
                         num_labels=2,
                         n_filters=100,
                         filter_sizes=[7,7,7,7],
                         dropout=0.1
                         )

# **Train the Model**

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)



In [ ]:
#optimizer应用AdamW，还有一个让learning rate逐渐变小的scheduler。
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=5e-5, 
                  eps=1e-8)

In [ ]:
epochs = 4

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
#加入f1_score来做model预测效果评估。

from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    
    preds_flat = np.argmax(preds, axis= 1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in value_dict.items()}

    preds_flat = np.argmax(preds, axis= 1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
#用来评估test的结果，会记录下每次预测的结果和正确率。

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        
        loss = outputs[0]
        logits = outputs[1]
        
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

# **Training**

In [ ]:
#这是model train 的部分，每个epoch后，都会把loss和f1_score显示。
#然后，每个epoch， train 完的model都会存下来。在test测试哪个更好。
torch.cuda.empty_cache()
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)

        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.5635201948704106
Validation loss: 0.5326700221712344
F1 Score (Weighted): 0.7420041316950227



Epoch 2
Training loss: 0.26020286178751156
Validation loss: 0.4005696224135773
F1 Score (Weighted): 0.8621337204423698



Epoch 3
Training loss: 0.08486022170046775
Validation loss: 0.5838524904276166
F1 Score (Weighted): 0.8707877032509213



Epoch 4
Training loss: 0.024175585196662543
Validation loss: 0.8412443518899185
F1 Score (Weighted): 0.8705050474505712



# **Test**

In [ ]:
model.load_state_dict(torch.load('data_volume/finetuned_BERT_epoch_3.model', map_location=torch.device('cuda')))

<All keys matched successfully>

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
value_dict={
    'fake':0,
    'true':1
}

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: fake
Accuracy: 1457/1711

Class: true
Accuracy: 1523/1711



In [ ]:
f1_score_func(predictions, true_vals)

0.8707877032509213